## Imports

In [ ]:
from LeWagon_FinalProject.data import *
from LeWagon_FinalProject.rapidapi import *
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#%matplotlib inline'
import pandas as pd

## Load data class

In [ ]:
rapid = Rapidapi(word_search='Bolsonaro', from_published_date='2019-01-01', to_published_date='2021-07-30')

In [ ]:
rapid.request_api()

In [ ]:
rapid.create_dataset()
rapid.df.head()

## Load data

In [ ]:
import requests
import pandas as pd
import os
import datetime
import time

In [ ]:
def date_validation(day, month, year):
    isValidDate = True
    try :
        datetime.datetime(int(year), int(month), int(day))   
    except ValueError :
        isValidDate = False
    return isValidDate

In [ ]:
search_dates = []
years = ['2018', '2019']
for year in years:
    for month in range(1,13):
        if month > 9:
            smonth = str(month)    
        else:
            smonth = '0'+str(month)
        for day in range(1,32):
            if day > 9:
                sday = str(day)    
            else:
                sday = '0'+str(day)
            if date_validation(day, month, year):
                search_dates.append(f'{year}-{smonth}-{sday}')
search_dates

In [ ]:
url = os.getenv('RAPIDAPI_URL')
print(url)
print(os.getenv('X_RAPIDAPI_KEY'))
print(os.getenv('X_RAPIDAPI_HOST'))

#querystring = {"q":"COVID","lang":"en","page":"1","page_size":"50","from":"2021-06-10","to":"2021-06-10"}
querystring = {"q":"COVID","lang":"en"}

headers = {
    'x-rapidapi-key': os.getenv('X_RAPIDAPI_KEY'),
    'x-rapidapi-host': os.getenv('X_RAPIDAPI_HOST')
    }

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.text)

In [ ]:
response_json = response.json()
#print(len(response_json['value']))
response_json

In [ ]:
ind_start = 0
ind_end = ind_start + 5
bstop = False
for i in range(0, len(search_dates)):
    print(search_dates[ind_start] + ' - ' + search_dates[ind_end])
    
    ind_start = ind_end + 1
    ind_end = ind_start + 5    
    if bstop:
        break
    if ind_start > (len(search_dates)-1):
        ind_start = (len(search_dates)-1)
    if ind_end > (len(search_dates)-1):
        ind_end = (len(search_dates)-1)
        bstop = True
    

In [ ]:
#news_columns = ['id', 'title', 'year', 'month', 'content']
news_columns = ['id', 'title', 'datepublished', 'content']
df_news = pd.DataFrame(columns=news_columns)

for search_date in search_dates:
    getNews = True
    pageNumber = 0
    numberNews = 0
    while getNews:
        pageNumber += 1
        url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/search/NewsSearchAPI"
        querystring = {"q":"COVID",
                       "pageNumber":str(pageNumber),
                       "pageSize":"50",
                       "autoCorrect":"true",
                       "fromPublishedDate": str(search_date),
                       "toPublishedDate": str(search_date)}
        headers = {
            'x-rapidapi-key': os.getenv('X_RAPIDAPI_KEY'),
            'x-rapidapi-host': os.getenv('X_RAPIDAPI_HOST')
            }
        response = requests.request("GET", url, headers=headers, params=querystring)
        response_json = response.json()
        #response_json.get('value', 0)
        if len(response_json['value']) > 0:
            for i in range(0, len(response_json['value'])):
                #datePublished = response_json['value'][i]['datePublished']
                new_news = {}
                new_news['id'] = response_json['value'][i]['id']
                new_news['title'] = response_json['value'][i]['title']
                #new_news['year'] = int(datePublished[0:4])
                #new_news['month'] = int(datePublished[5:7])
                new_news['datepublished'] = response_json['value'][i]['datePublished']
                new_news['content'] = response_json['value'][i]['body']
                df_news = df_news.append(new_news, ignore_index=True)
                numberNews += 1
                time.sleep(1)
        else:
            getNews = False        
    print(f'{search_date} : number news {numberNews}')

In [ ]:
news_columns = ['id', 'title', 'datepublished', 'content']
df_news = pd.DataFrame(columns=news_columns)

ind_start = 0
ind_end = ind_start + 5
bstop = False
for k in range(0, len(search_dates)):
    print(search_dates[ind_start] + ' - ' + search_dates[ind_end])
    
    getNews = True
    pageNumber = 0
    numberNews = 0
    while getNews:
        pageNumber += 1
        url = "https://free-news.p.rapidapi.com/v1/search"
        querystring = {"q":"COVID",
                       "pageNumber":str(pageNumber),
                       "pageSize":"50",
                       "autoCorrect":"true",
                       "fromPublishedDate": str(search_dates[ind_start]),
                       "toPublishedDate": str(search_dates[ind_end])}
        headers = {
            'x-rapidapi-key': os.getenv('X_RAPIDAPI_KEY'),
            'x-rapidapi-host': os.getenv('X_RAPIDAPI_HOST')
            }
        response = requests.request("GET", url, headers=headers, params=querystring)
        response_json = response.json()
        #response_json.get('value', 0)
        if len(response_json['value']) > 0:
            for i in range(0, len(response_json['value'])):
                #datePublished = response_json['value'][i]['datePublished']
                new_news = {}
                new_news['id'] = response_json['value'][i]['id']
                new_news['title'] = response_json['value'][i]['title']
                #new_news['year'] = int(datePublished[0:4])
                #new_news['month'] = int(datePublished[5:7])
                new_news['datepublished'] = response_json['value'][i]['datePublished']
                new_news['content'] = response_json['value'][i]['body']
                df_news = df_news.append(new_news, ignore_index=True)
                numberNews += 1
                time.sleep(1)
        else:
            getNews = False       
    print(f'{search_date} : number news {numberNews}')
    
    ind_start = ind_end + 1
    ind_end = ind_start + 5    
    if bstop:
        break
    if ind_start > (len(search_dates)-1):
        ind_start = (len(search_dates)-1)
    if ind_end > (len(search_dates)-1):
        ind_end = (len(search_dates)-1)
        bstop = True

In [ ]:
def remove_punctuation(text):
    text = text.replace('\r\n', ' ')
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')
    return text

In [ ]:
df_news['title'] = df_news['title'].apply(remove_punctuation)
df_news['content'] = df_news['content'].apply(remove_punctuation)
#df_news.to_csv('../raw_data/proj_final/brazil_elections.csv', header=True, index=True, encoding='utf-8')

In [ ]:
print(df_news.shape)
df_news.to_csv('../raw_data/proj_final/COVID.csv', header=True, index=True, encoding='utf-8')
#df_news['year'].min()
#df_news[['year', 'month', 'title']].groupby(by=['year', 'month']).count()
df_news

## Generate Data

In [ ]:
from LeWagon_FinalProject.topic_generate_graph import GenerateGraphData
gen = GenerateGraphData(file_path = '../raw_data/proj_final/Bolsonaro/')
gen.create_year_month_datasets(base_dataset_path = '../raw_data/proj_final/Bolsonaro_news.csv', min_topic_size=10)

## Prepare Data

In [ ]:
from LeWagon_FinalProject.topic_prepare_graph import PrepareGraphDataset

prep = PrepareGraphDataset(file_path = '../raw_data/proj_final/Bolsonaro/')
prep.generate_graph_dataset(keep_documents_no_topic=False, similarity_threshold=0.6)
prep.generate_graph_dataset(keep_documents_no_topic=True, similarity_threshold=0.6)

## News API Key

In [ ]:
# https://newsapi.org/docs/client-libraries/python
# https://free-docs.newscatcherapi.com/#introduction
# https://free-docs.newscatcherapi.com/#a-list-of-great-projects-built-with-free-news-api

In [ ]:
from newsapi import NewsApiClient
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
#print(os.getenv('NEWS_API_KEY'))

In [ ]:
# Init
newsapi = NewsApiClient(api_key=os.getenv('NEWS_API_KEY'))

# /v2/top-headlines
'''
top_headlines = newsapi.get_top_headlines(q='bitcoin',
                                          sources='bbc-news,the-verge',
                                          category='business',
                                          language='en',
                                          country='us')
'''

# /v2/everything
# health
# vaccines
all_articles = newsapi.get_everything(q='vaccines',
                                      from_param='2021-07-02',
                                      to='2021-07-31',
                                      language='en',
                                      sort_by='relevancy',
                                      page=1)

# /v2/top-headlines/sources
#sources = newsapi.get_sources()

In [ ]:
all_articles

In [ ]:
getNews = True
pageNumber = 1
all_articles_list = []
while getNews:
    print(pageNumber)
    all_articles = newsapi.get_everything(q='health',
                                      from_param='2021-07-02',
                                      to='2021-07-31',
                                      language='en',
                                      sort_by='relevancy',
                                      page=pageNumber)
    pageNumber += 1
    all_articles_list += all_articles['articles']
    if len(all_articles['articles']) <= 0:
        getNews = False

In [ ]:
len(all_articles_list)

In [ ]:
len(all_articles['articles'])

In [ ]:
len(all_articles['articles'][1]['content'])

## Free News api

In [ ]:
# https://rapidapi.com/newscatcher-api-newscatcher-api-default/api/free-news?utm_source=accept-invitation&utm_medium=email&utm_campaign=Free%20News

In [13]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

url = os.getenv('RAPIDAPI_URL')
print(url)

#querystring = {"q":"COVID","lang":"en", "from":"2019-01-01", "to":"2020-12-31"}
querystring = {"q":"COVID","lang":"en", "page_size": 25, "page": 10}

headers = {
    'x-rapidapi-key': os.getenv('X_RAPIDAPI_KEY'),
    'x-rapidapi-host': os.getenv('X_RAPIDAPI_HOST')
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

https://free-news.p.rapidapi.com/v1/search
{"status":"ok","total_hits":10000,"page":10,"total_pages":400,"page_size":25,"articles":[{"title":"Teenagers in UK to receive COVID vaccine","author":"news.com.au \u2014 australia\u2019s leading news site","published_date":"2021-08-05 08:04:43","published_date_precision":"timezone unknown","link":"https://www.news.com.au/world/teenagers-in-uk-to-receive-covid-vaccine/video/e85ae30fc883ea061705e49d0e0424ee","clean_url":"news.com.au","summary":"The United Kingdom will begin offering a first dose of the COVID-19 vaccine to 16 and 17 year olds, with the rollout expected to begin this month ahead of students' return to school. A second jab will be provided once more data has been collected. The UK's medicines authority has suggested children as young as 12 could eventually be offered the vaccine but will not be advised as of yet.","rights":"news.com.au","rank":570,"topic":"news","country":"AU","language":"en","authors":["news.com.au \u2014 australi

In [32]:
response_json = response.json()
#print(len(response_json['value']))
for i in range(0,25):
    #print(i)
    if response_json['articles'][i]['summary'] != None:
        print(len(response_json['articles'][i]['summary']))

389
0
500
500
0
500
500
217
500
500
500
500
500
500
500
166
500
500
0
500
500
500
500
500
